In [ ]:
%%sh
# export predict service script to gcs
gsutil cp /home/jovyan/work/<path-to-service>/predict-service.py gs://jupyter-storage/<path-in-gcs>/predict-service.py

In [ ]:
from kubernetes import client, config

## CREATE PREDICT SERVICE

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()

service = client.V1Service()
service.metadata = client.V1ObjectMeta(name="predict")

spec = client.V1ServiceSpec()
spec.type="LoadBalancer"
spec.selector = {"component" : "predict"}

ports = client.V1ServicePort(port=5000, target_port=5000)
spec.ports = [ports]

service.spec = spec

v1.create_namespaced_service(namespace="kubeyard", body=service)

## CREATE PREDICT POD

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()

pod = client.V1Pod()
spec = client.V1PodSpec()
pod.metadata = client.V1ObjectMeta(name="predict", labels={"component" : "predict"})

container = client.V1Container()
container.image = "registry.inovex.de:4567/inovex-kubeyard/docker-jupyter:latest"
container.args = ["/bin/bash", "-c", "gsutil cp gs://jupyter-storage/<path-in-gcs>/predict-service.py . && ipython -- predict-service.py"]
container.name = "predict"
container.ports = [client.V1ContainerPort(container_port=5000)]

spec.containers = [container]
pod.spec = spec

v1.create_namespaced_pod(namespace="kubeyard", body=pod)

## GET SERVICE ENDPOINT

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()
response = v1.read_namespaced_service(name="predict", namespace="kubeyard")

print(response.status.load_balancer.ingress[0].ip + ":" + str(response.spec.ports[0].port))

## DELETE PREDICT SERVICE AND POD

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()

#v1.delete_namespaced_pod(name="predict", namespace="kubeyard", body=client.V1DeleteOptions())
v1.delete_namespaced_service(name="predict", namespace="kubeyard")